In [1]:
import SimpleITK as sitk
import tkinter as tk
from tkinter import filedialog
import cv2 as cv
import numpy as np
import logging

import sys
# Add the directory to the sys.path
sys.path.append('D:/EmpaData/COFO - HP/03 - Github/dvpt-at-EMPA/HappyBisons/image analysis')
# Now you can import modules from that directory
import tools.CT_modification_functions as CT_modification_functions
import tools.CT_visualization_functions as CT_visualization_functions
import tools.CT_visualization_window as CT_visualization_window
%matplotlib qt

In [2]:
root = tk.Tk()
# Prompt the user to select a file using a file explorer
input_file_path = filedialog.askopenfilename() # this is the mhd file contnaing the whole image, no filter, no segmentation
print(input_file_path)
root.withdraw()


D:/EmpaData/COFO - HP/01 PhD research/202409 - Belgium skulls/WP1 - scans of horn/scan all/bel-001/mhd/skull c326i.mhd


''

In [3]:
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(level=logging.DEBUG)

CT_visualization_window.open_window(sitk.ReadImage(input_file_path))
#blurred_image = sitk.SmoothingRecursiveGaussian(map_image, sigma=0.3)


INFO:root:starting open window function
INFO:root:For memory saving, your images will be converted to UINT8
DEBUG:root:your image takes 33.695529048 gigabytes
DEBUG:root:your image has been reduced to 8.423882262 gigabytes
INFO:root:starting window setup function
DEBUG:root:converting grey scale image to rgb
DEBUG:root:rgb image takes 25.271646786 gigabytes
DEBUG:root:conversion succesfull in -25.42760682106018 seconds.
DEBUG:root:opening figure
DEBUG:matplotlib.font_manager:findfont: Matching sans\-serif:style=normal:variant=normal:weight=normal:stretch=normal:size=10.0.
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='c:\\Users\\cofo\\.conda\\envs\\medical_imaging\\Lib\\site-packages\\matplotlib\\mpl-data\\fonts\\ttf\\DejaVuSansMono-Bold.ttf', name='DejaVu Sans Mono', style='normal', variant='normal', weight=700, stretch='normal', size='scalable')) = 10.335
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='c:\\Users\\cofo\\.conda\\envs\\medical_imaging\\Lib

opening new histogram window


DEBUG:matplotlib.font_manager:findfont: Matching sans\-serif:style=normal:variant=normal:weight=normal:stretch=normal:size=12.0.
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='c:\\Users\\cofo\\.conda\\envs\\medical_imaging\\Lib\\site-packages\\matplotlib\\mpl-data\\fonts\\ttf\\DejaVuSansMono-Bold.ttf', name='DejaVu Sans Mono', style='normal', variant='normal', weight=700, stretch='normal', size='scalable')) = 10.335
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='c:\\Users\\cofo\\.conda\\envs\\medical_imaging\\Lib\\site-packages\\matplotlib\\mpl-data\\fonts\\ttf\\cmsy10.ttf', name='cmsy10', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='c:\\Users\\cofo\\.conda\\envs\\medical_imaging\\Lib\\site-packages\\matplotlib\\mpl-data\\fonts\\ttf\\STIXSizThreeSymReg.ttf', name='STIXSizeThreeSym', style='normal', variant='normal', weight=400, stretch='normal', si

opening sliders window


INFO:root:changing colorset to grey-scale
DEBUG:root:converting grey scale image to rgb
DEBUG:root:rgb image takes 25.271646786 gigabytes
DEBUG:root:conversion succesfull in -27.047041177749634 seconds.
DEBUG:root:new color scale applied in -36.516069412231445 seconds.


opening new histogram window
closing histogram window


INFO:root:window setup function succesfull


array([], dtype=float64)

In [ ]:


# Clear any existing logging handlers
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(level=logging.DEBUG)

# Function to divide two numbers with logging
def divide(a, b):
    logging.debug(f"Entering divide function with arguments: a={a}, b={b}")
    try:
        result = a / b
        logging.info(f"Division successful: {a} / {b} = {result}")
        return result
    except ZeroDivisionError as e:
        logging.error("Attempted to divide by zero")
        return None

# Test the function with various inputs
divide(10, 2)    # Normal case
divide(10, 0)    # Error case


DEBUG:root:Entering divide function with arguments: a=10, b=2
INFO:root:Division successful: 10 / 2 = 5.0
DEBUG:root:Entering divide function with arguments: a=10, b=0
ERROR:root:Attempted to divide by zero


In [28]:
map_image = sitk.ReadImage(input_file_path)
one_image = sitk.GetArrayFromImage(map_image)[1200,:,:]
#CT_visualization_window.open_window(map_image)
#print(one_image.GetPixelIDTypeAsString())


one_image = cv.normalize(one_image, None, 0, 255, cv.NORM_MINMAX, cv.CV_8U)
cv.imshow("one_image", one_image)




In [29]:
kernel = np.ones((3, 3), np.uint8)

binary_image = np.zeros_like(one_image, dtype=np.uint8)
binary_image[(one_image > 10) & (one_image < 130)] = 255

binary_image = cv.erode(binary_image,kernel,iterations=2)
binary_image = cv.dilate(binary_image,kernel,iterations=3)
binary_image = cv.erode(binary_image,kernel,iterations=1)
cv.imshow("Binary Image", binary_image)

In [31]:
import cv2 as cv
import numpy as np
"""
def watershed_algorithm(single_image):
    single_image = sitk.GetArrayFromImage(single_image)
    kernel = np.ones((3,3),np.uint8)
    # sure background area
    sure_bg = cv.dilate(single_image,kernel,iterations=2)
    # Finding sure foreground area
    temp = cv.distanceTransform(single_image,cv.DIST_L2,3)
    sure_fg = cv.threshold(temp,0.2*dist_transform.max(),255,0)
    del temp
    # Finding unknown region
    unknown = cv.subtract(sure_bg,sure_fg)
    # markers
    markers = cv.connectedComponents(sure_fg)
    markers = markers+1     # Add one to all labels so that sure background is not 0, but 1
    markers[unknown==255] = 0 # Now, mark the region of unknown with zero
    # results
    markers = cv.watershed(single_image,markers)
    single_image[markers == -1] = [255,0,0]
    return single_image
"""


def watershed_algorithm(single_image, binary_image):
    ''' 
    Lets you use a watershed algorithm to color edges of zones (touching or not) black.
    
    Inputs: - single image: np.array ; is used for the "topography of your zones
            - binary_image: np.array ; is used to define where the objects are

    Outputs: - modified input image
    '''
    # Define a kernel for morphological operations
    kernel = np.ones((3, 3), np.uint8)

    # Convert SimpleITK image to NumPy array UINT8
    #single_image = sitk.GetArrayFromImage(single_image)
    single_image = cv.normalize(single_image, None, 0, 255, cv.NORM_MINMAX, cv.CV_8U)
    binary_image = cv.normalize(binary_image, None, 0, 255, cv.NORM_MINMAX, cv.CV_8U)

    # Dilate to get sure background area
    sure_bg = np.copy(binary_image)
    #sure_bg[single_image>threshold_level_max*1.25] = 0
    sure_bg = cv.dilate(sure_bg, kernel, iterations=4).astype(np.uint8)
    
    
    # Apply distance transform and threshold to get sure foreground
    dist_transform = cv.distanceTransform(binary_image, cv.DIST_L2, 3)
    _, sure_fg = cv.threshold(dist_transform, 0.1 * dist_transform.max(), 255, 0) #the sure foreground appear as 255
    sure_fg = np.uint8(sure_fg)  # Convert sure foreground to uint8 type for compatibility

    # Identify unknown regions by subtracting sure foreground from background
    unknown = cv.subtract(sure_bg, sure_fg)     #the unknown regions appears as 255

    # Marker labeling
    number_of_zones, connection_markers = cv.connectedComponents(sure_fg)
    # Add one to ensure background is 1, not 0
    connection_markers = connection_markers + 1
    # Mark the unknown regions as 0
    connection_markers[unknown == 255] = 0

    #connection_markers_display = cv.normalize(connection_markers, None, 0, 65535, cv.NORM_MINMAX).astype(np.uint16)
    connection_markers_display = np.copy(connection_markers).astype(np.uint16)
    connection_markers_display = cv.normalize(connection_markers, None, 65535/5, 65535, cv.NORM_MINMAX, cv.CV_16U)
    connection_markers_display[connection_markers_display == connection_markers_display.min()] = 0

    # Ensure single_image is in color (3-channel) format for watershed
    single_image_color = cv.cvtColor(single_image, cv.COLOR_GRAY2BGR)

    # Apply watershed algorithm
    markers = cv.watershed(single_image_color, connection_markers)
    markers = cv.normalize(markers, None, 0, 65535, cv.NORM_MINMAX, cv.CV_32S)

    # Mark boundaries in red
    single_image[markers == 0] = 255

    # Uncomment these lines if running in a local environment with OpenCV GUI support
    
    cv.imshow("Binary Image", binary_image)
    cv.waitKey(0)
    cv.imshow("Sure Background", sure_bg)
    cv.imshow("Sure Foreground", sure_fg)
    cv.imshow("Unknown Regions", unknown)
    
    cv.imshow("connection markers", connection_markers_display)
    cv.waitKey(0)
    cv.imshow("Markers", cv.normalize(markers, None, 0, 65535, cv.NORM_MINMAX, cv.CV_16U))
    cv.waitKey(0)
    cv.imshow("single_image", single_image)
    
    cv.waitKey(0)
    cv.destroyAllWindows()
    
    return single_image



result = watershed_algorithm(one_image, binary_image)



In [5]:
import cv2 as cv
import numpy as np

# Create a simple binary image with three distinct blobs
binary_image = np.array([[0, 0, 0, 0, 255],
                         [0, 255, 255, 0, 0],
                         [0, 255, 255, 0, 255],
                         [0, 0, 0, 0, 255]], dtype=np.uint8)

# Apply connectedComponents to label the blobs
print(np.invert(binary_image))
num_labels, labels = cv.connectedComponents(binary_image)

print("Number of labels:", num_labels)
print("Labels matrix:\n", labels)
# Normalize the labels for visualization as an 8-bit image
# Scale labels to the 0-255 range for display purposes
labels_display = cv.normalize(labels, None, 0, 255, cv.NORM_MINMAX).astype(np.uint8)

# Display the labeled components
cv.imshow('Labeled Components', labels_display)


[[255 255 255 255   0]
 [255   0   0 255 255]
 [255   0   0 255   0]
 [255 255 255 255   0]]
Number of labels: 4
Labels matrix:
 [[0 0 0 0 2]
 [0 1 1 0 0]
 [0 1 1 0 3]
 [0 0 0 0 3]]


In [9]:


out = CT_visualization_functions.select_pixels(map_image)

image_array = sitk.GetArrayFromImage(map_image)

for row in out:
    image_array[row[2],row[1],row[0]] = 0 #numpy array is a pain


CT_visualization_functions.browse_images(sitk.GetImageFromArray(image_array))


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


Clicked at coordinates: x=203.3477825464949, y=430.3340486409156, in slice 687, Pixel value: 99
Clicked at coordinates: x=318.4684310920361, y=225.36313781592764, in slice 687, Pixel value: 255


In [4]:
root = tk.Tk()
# Prompt the user to select a file using a file explorer
input_file_path_1 = filedialog.askopenfilename() # this is the mhd file contnaing the whole image, no filter, no segmentation
root.withdraw()
root = tk.Tk()
# Prompt the user to select a file using a file explorer
input_file_path_2 = filedialog.askopenfilename() # this is the mhd file contnaing the whole image, no filter, no segmentation
root.withdraw()

''

In [5]:
image_1 = sitk.ReadImage(input_file_path_1)
image_2 = sitk.ReadImage(input_file_path_2)

CT_visualization_functions.browse_blended_images(image_1,image_2,True)

